<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/AGENTIC-MODULE6-Perception_Precompute.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

# ============================================================
# Fault Classification Pipeline
# ============================================================

############PASTE ADAPTIVE WINDOW HERE - so everything is in one file - later, we can import as a package#####################


# ====== AdaptiveWindowAgent ======
# =====================================================
# AdaptiveWindowAgent (improved version)
# =====================================================
# agents/adaptive_window_agent.py
import numpy as np
import pandas as pd
import pickle
import json
import os
from collections import deque
from typing import Dict, Any
import datetime as dt
import logging
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor  # ✅ MOVED TO TOP
from statsmodels.tsa.vector_ar.var_model import VAR
import keras
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime

logger = logging.getLogger(__name__)

import sqlite3
import json
from datetime import datetime
import json


class EventStore:
    def __init__(self, db_path="event_store.db"):
        self.conn = sqlite3.connect(db_path)
        self._init_tables()

    def _init_tables(self):
        c = self.conn.cursor()
        c.execute("""
            CREATE TABLE IF NOT EXISTS events (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                timestamp TEXT,
                event_type TEXT,
                packet_json TEXT,
                expert_json TEXT,
                human_json TEXT
            )
        """)
        self.conn.commit()

    def save_event(self, event_type, packet, expert=None, human=None):
        c = self.conn.cursor()
        c.execute(
            "INSERT INTO events(timestamp,event_type,packet_json,expert_json,human_json) VALUES (?,?,?,?,?)",
            (
                datetime.now().isoformat(),
                event_type,
                json.dumps(packet, default=str),
                json.dumps(expert, default=str) if expert else None,
                json.dumps(human, default=str) if human else None,
            )
        )
        self.conn.commit()

    def fetch_recent(self, limit=100):
        c = self.conn.cursor()
        c.execute("SELECT packet_json, expert_json, human_json FROM events ORDER BY id DESC LIMIT ?", (limit,))
        return [json.loads(row[0]) for row in c.fetchall()]

######################################################################################
# DECISION POLICY CONFIGS-----------------------------------------------------------
######################################################################################'

# =====================================================
# DECISION POLICY (SEPARATED FROM LOGIC)
# =====================================================

BASE_POLICY = {
    # --- detection fusion ---
    "w_sensor": 0.60,
    "w_window": 0.40,

    # --- drift fusion ---
    "w_fdi": 0.50,
    "w_wdi": 0.25,
    "w_sensor_drift": 0.25,

    # --- failure prediction ---
    "w_fault": 0.70,
    "w_warn": 0.15,
    "w_det_context": 0.15,

    # --- thresholds ---
    "detection_threshold": 0.50,
    "drift_threshold": 0.35,
    "failure_threshold": 0.50,
    "failure_critical_threshold": 0.80,

    # --- alert mapping ---
    "alert_low": 0.35,
    "alert_med": 0.55,
    "alert_high": 0.75,
}


#########################################################################
# Window Agent - Global Context or Global Predictive Context
#########################################################################

class AdaptiveWindowAgent:
    """
    Adaptive Window Agent:
    - Predicts window size using MLP
    - Evaluates forecast with RF/persistence
    - Computes:
        * FDS: Forecast Deviation Score (normalized error)
        * FDI: Forecast Drift Index (JSD over FDS distribution)
        * WSS: Window Shift Score (normalized window size)
        * WDI: Window Drift Index (JSD over window size distribution)
    - Detects anomaly (local) + drift (regime) events.
    """

    def __init__(self, agent_id="adaptive_window_agent",
                 model_path=None, checkpoint_path=None):
        self.agent_id = agent_id
        self.model_path = model_path or "/content/drive/MyDrive/PHD/2025/DGRNet-MLP-Versions/METROPM_MLP_model_10Sec.keras"

        self.baseline_path = "/content/drive/MyDrive/PHD/2025/DGRNet-MLP-Versions/METROPM_MLP_baseline.pkl"
        self.checkpoint_path = checkpoint_path

        # --------------------------------------------------
        # Core model
        # --------------------------------------------------
        self.model = None
        self.transformer = StandardScaler()
        self.transformer_fitted = False
        self.is_model_loaded = False

        # Baseline error stats (median / MAD) – filled from baseline file
        self.rolling_stats = {
            'median': 0.0,
            'mad': 1.0,
        }

        # --------------------------------------------------
        # Metrics memory
        # --------------------------------------------------
        # Raw error
        self.error_memory = deque(maxlen=300)     # long-term errors
        self.recent_errors = deque(maxlen=50)     # kept for backward compat (not central now)

        # FDS (Forecast Deviation Score) history
        self.fds_memory = deque(maxlen=300)
        self.recent_fds = deque(maxlen=50)

        # Window history
        self.window_memory = deque(maxlen=300)
        self.recent_windows = deque(maxlen=50)

        # Last-step metrics (for returning)
        self.last_fds = 0.0
        self.last_fdi = 0.0
        self.last_wss = 0.0
        self.last_wdi = 0.0

        # Baseline distributions
        self.baseline_errors = None
        self.baseline_fds = None
        self.baseline_windows = None
        self.window_mean = 50.0    # a reasonable mid value
        self.window_std = 10.0     # non-zero, avoids div-by-zero

        # Optional histogram bins stored in baseline
        self.window_hist_bins = None
        self.window_hist_counts = None

        # Debug flag (OFF by default)
        self.debug = False

        # --------------------------------------------------
        # Anomaly / drift settings
        # --------------------------------------------------
        self.threshold_k = 3.0
        self.anomaly_cooldown = 0
        self.anomaly_cooldown_steps = 5

        # Drift detection
        self.drift_threshold = 0.25          # for FDI (JSD over FDS)
        self.window_drift_threshold = 0.20   # for WDI (JSD over window sizes)
        self.consecutive_drift_votes = 0
        self.drift_cooldown = 0
        self.drift_votes_required = 10
        self.drift_cooldown_steps = 100

        # --------------------------------------------------
        # Retraining buffer (unchanged)
        # --------------------------------------------------
        self.performance_stats = {
            'total_predictions': 0,
            'avg_mse': 0.0,
            'avg_mae': 0.0,
            'last_retrain_time': None,
            'drift_events': 0,
            'anomaly_events': 0,
            'retraining_events': 0
        }

        self.retraining_data = {
            'x_buffer': deque(maxlen=10000),
            'y_buffer': deque(maxlen=10000)
        }

        # --------------------------------------------------
        # Prediction history (for reporting)
        # --------------------------------------------------
        self.prediction_history = deque(maxlen=1000)
        self.mse_history = deque(maxlen=200)
        self.mae_history = deque(maxlen=200)

        # --------------------------------------------------
        # Load baseline (errors + windows)
        # --------------------------------------------------
        self._load_baseline()

        # --------------------------------------------------
        # Load model last
        # --------------------------------------------------
        self.load_model()
        print(f"AdaptiveWindowAgent {self.agent_id} initialized")

        # --------------------------------------------------
        # Load NSP (Next-Step Predictor)
        # --------------------------------------------------
        self.nsp_model_path = "/content/drive/MyDrive/PHD/2025/NSP_LSTM_next_step.keras"
        self.nsp_model = keras.models.load_model(self.nsp_model_path)
        print("✅ Loaded NSP LSTM next-step predictor")

    # =================== BASELINE LOADING ===================

    def _load_baseline(self):
        """
        Load baseline stats:
          - baseline_errors, median, mad
          - baseline_windows, window_mean, window_std
          - optional histogram bins/counts for windows
        """
        if os.path.exists(self.baseline_path):
            with open(self.baseline_path, "rb") as f:
                base = pickle.load(f)

            # Error baseline
            self.baseline_errors = np.array(base["baseline_errors"])
            self.rolling_stats["median"] = base["median"]
            self.rolling_stats["mad"] = base["mad"]

            # Precompute baseline FDS distribution
            med = self.rolling_stats["median"]
            mad = self.rolling_stats["mad"] if self.rolling_stats["mad"] > 0 else 1e-6
            self.baseline_fds = (self.baseline_errors - med) / (mad + 1e-8)

            # Window baseline (may or may not exist)
            if "baseline_windows" in base:
                self.baseline_windows = np.array(base["baseline_windows"])
                self.window_mean = float(base.get("window_mean", np.mean(self.baseline_windows)))
                self.window_std = float(base.get("window_std", np.std(self.baseline_windows) + 1e-8))
                self.window_hist_bins = np.array(base.get("window_hist_bins", [])) if "window_hist_bins" in base else None
                self.window_hist_counts = np.array(base.get("window_hist_counts", [])) if "window_hist_counts" in base else None
            else:
                self.baseline_windows = None
                self.window_mean = 0.0
                self.window_std = 1.0
                self.window_hist_bins = None
                self.window_hist_counts = None

            print("✅ Loaded baseline error + window distribution.")
            print(f"   Error median={self.rolling_stats['median']:.6f}, MAD={self.rolling_stats['mad']:.6f}")
            if self.baseline_windows is not None:
                print(f"   Window mean={self.window_mean:.3f}, std={self.window_std:.3f}")
        else:
            print("⚠️ No baseline found. Using live history only.")
            self.baseline_errors = None
            self.baseline_fds = None
            self.baseline_windows = None

    # =================== MODEL LOADING ===================

    def load_model(self):
        try:
            if os.path.exists(self.model_path):
                self.model = keras.models.load_model(self.model_path)
                self.is_model_loaded = True
                print(f"✅ Loaded MLP model from {self.model_path}")

                # Try to load transformer
                transformer_path = self.model_path.replace('.keras', '_transformer.pkl')
                if os.path.exists(transformer_path):
                    with open(transformer_path, 'rb') as f:
                        self.transformer = pickle.load(f)
                    self.transformer_fitted = True
                else:
                    # Fit transformer from true window labels
                    y_original = np.load(
                        "/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/generated-data-true-window2.npy"
                    )
                    self.transformer.fit(y_original.reshape(-1, 1))
                    self.transformer_fitted = True
                    with open(transformer_path, 'wb') as f:
                        pickle.dump(self.transformer, f)
                    print("⚠️ No transformer found, fitted a new one.")
            else:
                print(f"❌ Model not found at {self.model_path}")
        except Exception as e:
            print(f"❌ Error loading model: {e}")

    # =================== FORECAST EVALUATION ===================

    def evaluate_forecast_performance(self, sequence_3d, predicted_window, n_future=1):
        try:
            seq = np.asarray(sequence_3d)
            T, F = seq.shape

            W = int(predicted_window)
            if W < 2:
                W = 2
            if W > T - n_future - 1:
                W = max(2, T - n_future - 1)

            # --- Prepare NSP input ---
            window = seq[-W:-n_future, :]   # shape (W-1, F)
            x = window[np.newaxis, ...]      # shape (1, W-1, F)

            # --- NSP prediction ---
            y_pred = self.nsp_model.predict(x, verbose=0)[0]  # (F,)
            y_true = seq[-n_future, :]                        # (F,)

            mse = float(np.mean((y_true - y_pred) ** 2))
            mae = float(np.mean(np.abs(y_true - y_pred)))

            return {
                "mse": mse,
                "mae": mae,
                "forecast_success": True,
                "actual_values": y_true.tolist(),
                "predicted_values": y_pred.tolist(),
                "window_size_used": W,
                "method": "NSP_LSTM"
            }

        except Exception as e:
            return {
                "mse": 9999.0,
                "mae": 9999.0,
                "forecast_success": False,
                "error": str(e),
                "method": "NSP_LSTM"
            }

    # =================== PERSISTENCE FALLBACK ===================

    def _persistence_forecast(self, seq, target_sensor_index, n_future):
        """
        Persistence fallback for RF evaluation.
        Last-value-carried-forward for target sensor.
        """
        try:
            seq = np.asarray(seq)
            if len(seq) < 2:
                return {
                    'mse': 9999,
                    'mae': 9999,
                    'forecast_success': False,
                    'error': 'Sequence too short',
                    'method': 'Persistence'
                }

            last_value = seq[-1, target_sensor_index]
            predicted_vals = [last_value]
            actual = [seq[-1, target_sensor_index]]

            mse = 0.0
            mae = 0.0

            return {
                'mse': mse,
                'mae': mae,
                'forecast_success': True,
                'actual_values': actual,
                'predicted_values': predicted_vals,
                'target_sensor_index': target_sensor_index,
                'method': 'Persistence',
                'note': 'persistence_fallback'
            }

        except Exception as e:
            return {
                'mse': 9999,
                'mae': 9999,
                'forecast_success': False,
                'error': str(e),
                'method': 'Persistence',
                'note': 'persistence_fallback_failed'
            }

    # =================== PREDICTION PIPELINE ===================

    def predict_window_size(self, feature_vector, sequence_3d):
        """
        Main entrypoint:
          - Predict window W_t
          - Evaluate forecast error e_t
          - Compute FDS (S_t) and WSS (Z_t)
          - Update drift/anomaly logic (FDI, WDI, events)
          - Return full metrics packet
        """
        if not self.is_model_loaded:
            return {'predicted_window': 20, 'error': "Model not loaded"}

        try:
            if feature_vector.ndim == 1:
                feature_vector = feature_vector.reshape(1, -1)

            pred_raw = self.model.predict(feature_vector, verbose=0)

            if self.transformer_fitted:
                predicted_window = int(round(self.transformer.inverse_transform(pred_raw)[0, 0]))
            else:
                predicted_window = int(round(pred_raw[0, 0]))

            # ----------------------------------------
            # WINDOW CLAMP — HARD SAFETY FIX
            # ----------------------------------------
            # Prevent negative, zero, or extreme window sizes
            predicted_window = max(2, predicted_window)        # lower bound

            # Forecast evaluation
            forecast_metrics = self.evaluate_forecast_performance(sequence_3d, predicted_window, n_future=1)

            fds = None
            wss = None

            if forecast_metrics.get("forecast_success", False):
                mse = forecast_metrics["mse"]
                mae = forecast_metrics["mae"]

                # Basic stats
                self.mse_history.append(mse)
                self.mae_history.append(mae)
                self.error_memory.append(mse)

                self.performance_stats['total_predictions'] += 1
                self.performance_stats['avg_mse'] = float(np.mean(self.mse_history))
                self.performance_stats['avg_mae'] = float(np.mean(self.mae_history))

                # ---------- Forecast Deviation Score (FDS) ----------
                baseline_median = self.rolling_stats["median"]
                baseline_mad = self.rolling_stats["mad"] if self.rolling_stats["mad"] > 0 else 1e-6
                fds = (mse - baseline_median) / (baseline_mad + 1e-8)
                fds = float(fds) if fds is not None and not np.isnan(fds) else 0.0


                self.last_fds = fds
                self.fds_memory.append(fds)
                self.recent_fds.append(fds)

                # ---------- Window Shift Score (WSS) ----------
                if self.baseline_windows is not None and self.window_std > 0:
                    wss = (predicted_window - self.window_mean) / (self.window_std + 1e-8)
                else:
                    wss = 0.0

                wss = float(wss)
                self.last_wss = wss
                self.window_memory.append(predicted_window)
                self.recent_windows.append(predicted_window)

            # Event (ANOMALY / DRIFT) + Drift indices
            event, sev, fdi, wdi = self._check_for_event()
            self.last_fdi = fdi
            self.last_wdi = wdi

            # Save history for reporting
            record = {
                'timestamp': dt.datetime.now(),
                'predicted_window': predicted_window,
                'forecast_metrics': forecast_metrics,
                'fds': self.last_fds,
                'wss': self.last_wss,
                'fdi': self.last_fdi,
                'wdi': self.last_wdi,
                'event_type': event,
                'severity': sev
            }
            self.prediction_history.append(record)

            return {
                'predicted_window': predicted_window,
                'forecast_metrics': forecast_metrics,
                'fds': self.last_fds,
                'fdi': self.last_fdi,
                'wss': self.last_wss,
                'wdi': self.last_wdi,
                'event_type': event,
                'severity': sev,
                'performance_stats': self.get_recent_performance()
            }
        except Exception as e:
            return {'predicted_window': 20, 'error': str(e)}

    # =================== EVENT LOGIC (ANOMALY + DRIFT) ===================

    def _check_for_event(self):
        """
        Event detection for the Adaptive Window Agent.

        - ANOMALY: deviation of last MSE from baseline (median + k * MAD)
        - DRIFT:
            * FDI: JSD between recent FDS distribution and baseline FDS
            * WDI: JSD between recent window distribution and baseline window distribution
        """
        # Require enough history
        if len(self.error_memory) < 30:
            return None, 0.0, None, None

        last_mse = float(self.error_memory[-1])
        live_errors = np.array(self.error_memory)

        # ---------- BASELINE STATS ----------
        if self.baseline_errors is not None and len(self.baseline_errors) > 10:
            base_errors = np.array(self.baseline_errors)
            baseline_median = np.median(base_errors)
            baseline_mad = np.median(np.abs(base_errors - baseline_median)) + 1e-8
        else:
            baseline_median = np.median(live_errors)
            baseline_mad = np.median(np.abs(live_errors - baseline_median)) + 1e-8

        # Update rolling_stats so other components can see latest baseline-ish values
        self.rolling_stats["median"] = baseline_median
        self.rolling_stats["mad"] = baseline_mad

        # ---------- LIVE STATS ----------
        live_median = np.median(live_errors)
        live_mad = np.median(np.abs(live_errors - live_median)) + 1e-8

        # ---------- ANOMALY THRESHOLD ----------
        baseline_threshold = baseline_median + self.threshold_k * baseline_mad
        live_threshold = live_median + self.threshold_k * live_mad

        anomaly_threshold = 0.8 * baseline_threshold + 0.2 * live_threshold

        is_anomaly = last_mse > anomaly_threshold

        if self.anomaly_cooldown > 0:
            self.anomaly_cooldown -= 1
            is_anomaly = False
        elif is_anomaly:
            self.anomaly_cooldown = self.anomaly_cooldown_steps

        if is_anomaly:
            severity = (last_mse - anomaly_threshold) / (baseline_mad + 1e-6)
            severity = float(severity)
            self.performance_stats["anomaly_events"] += 1
            if self.debug:
                print(f"[ANOMALY] mse={last_mse:.6f}, thr={anomaly_threshold:.6f}, sev={severity:.3f}")
            return "ANOMALY", severity, 0.0, 0.0

        # ---------- DRIFT (FDI + WDI) ----------
        fdi = None
        wdi = None

        # FDI: JSD over FDS distribution
        if self.baseline_fds is not None and len(self.recent_fds) >= 30:
            base_fds = np.asarray(self.baseline_fds)
            recent_fds = np.asarray(self.recent_fds)

            hist_base, bins = np.histogram(base_fds, bins=25, density=True)
            hist_recent, _ = np.histogram(recent_fds, bins=bins, density=True)

            hist_base = hist_base / (hist_base.sum() + 1e-12)
            hist_recent = hist_recent / (hist_recent.sum() + 1e-12)

            fdi = float(jensenshannon(hist_base + 1e-12, hist_recent + 1e-12))

        # WDI: JSD over window-size distribution
        if self.baseline_windows is not None and len(self.recent_windows) >= 30:
            base_win = np.asarray(self.baseline_windows)
            recent_win = np.asarray(self.recent_windows)

            hist_w_base, bins_w = np.histogram(base_win, bins=20, density=True)
            hist_w_recent, _ = np.histogram(recent_win, bins=bins_w, density=True)

            hist_w_base = hist_w_base / (hist_w_base.sum() + 1e-12)
            hist_w_recent = hist_w_recent / (hist_w_recent.sum() + 1e-12)

            wdi = float(jensenshannon(hist_w_base + 1e-12, hist_w_recent + 1e-12))

        # Decide drift if either index is high
        is_drift_fdi = fdi is not None and fdi > self.drift_threshold
        is_drift_wdi = wdi is not None and wdi > self.window_drift_threshold

        is_drift = is_drift_fdi or is_drift_wdi

        if self.drift_cooldown > 0:
            self.drift_cooldown -= 1
            is_drift = False
        else:
            if is_drift:
                self.consecutive_drift_votes += 1
            else:
                self.consecutive_drift_votes = 0

        if self.consecutive_drift_votes >= self.drift_votes_required:
            self.consecutive_drift_votes = 0
            self.drift_cooldown = self.drift_cooldown_steps
            self.performance_stats["drift_events"] += 1
            if self.debug:
                print(f"[DRIFT] FDI={fdi:.4f} WDI={wdi:.4f}")
            fdi = float(fdi) if fdi is not None else 0.0
            wdi = float(wdi) if wdi is not None else 0.0
            return "DRIFT", fdi, fdi, wdi

        # Make safe for printing
        fdi = float(fdi) if fdi is not None else 0.0
        wdi = float(wdi) if wdi is not None else 0.0

        return None, 0.0, fdi, wdi


    # =================== HELPERS ===================

    def get_recent_performance(self):
        all_preds = list(self.prediction_history)

        successful_predictions = [
            p for p in all_preds
            if p.get('forecast_metrics', {}).get('forecast_success', False)
        ]

        return {
            'total_predictions': len(all_preds),
            'successful_predictions': len(successful_predictions),
            'success_rate': len(successful_predictions) / max(len(all_preds), 1),
            'drift_events': self.performance_stats['drift_events'],
            'anomaly_events': self.performance_stats['anomaly_events'],
            'retraining_events': self.performance_stats['retraining_events'],
            'recent_mse': float(np.mean(list(self.mse_history)[-10:])) if self.mse_history else 0,
            'avg_mse': float(np.mean(self.mse_history)) if self.mse_history else 0,
            'recent_mae': float(np.mean(list(self.mae_history)[-10:])) if self.mae_history else 0,
            'avg_mae': float(np.mean(self.mae_history)) if self.mae_history else 0,
            'transformer_fitted': self.transformer_fitted,
            'last_fdi': self.last_fdi,
            'last_wdi': self.last_wdi,
        }

    def save_performance_state(self, filepath: str):
        """Save performance statistics + prediction history to JSON"""
        try:
            state = {
                'performance_stats': self.performance_stats.copy(),
                'prediction_history': list(self.prediction_history)[-100:],
                'mse_history': list(self.mse_history),
                'mae_history': list(self.mae_history),
                'transformer_fitted': self.transformer_fitted
            }
            with open(filepath, 'w') as f:
                json.dump(state, f, indent=2, default=str)
            print(f"✅ Performance state saved to {filepath}")
        except Exception as e:
            print(f"❌ Failed to save performance state: {e}")


#===============================================================================================================================================
###  SENSOR AGENTS - INDIVIDUAL AND MASTER
#--------------------------------------==========================================================================================================
import numpy as np
import pickle
import os
from collections import deque
from datetime import datetime
from typing import Dict, List, Tuple
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

# ML libraries
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from scipy import stats
from scipy.spatial.distance import jensenshannon

# Deep learning
try:
    from tensorflow.keras.models import load_model
    KERAS_AVAILABLE = True
except ImportError:
    KERAS_AVAILABLE = False

from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    roc_auc_score,
    average_precision_score,
    precision_recall_curve,
    roc_curve
)


# =====================================================
# ROBUST SENSOR AGENT - Observes ONE sensor with AE model
# =====================================================


# =====================================================
# ROBUST SENSOR AGENT - Observes ONE sensor with AE model
# =====================================================

class RobustSensorAgent:
    """
    Robust Sensor Agent for ONE sensor with advanced anomaly & drift detection.

    Loads pretrained AE model + metadata (scaler, baseline errors, rolling stats).
    Computes anomaly score via reconstruction error, applies adaptive thresholding,
    drift detection, and outputs robust anomaly/drift/retrain flags.
    """

    def __init__(self,
                 sensor_id: int,
                 model_path: str = None,
                 window_length: int = 10, #K
                 memory_size: int = 1000,
                 threshold_k: float = 2.0,
                 drift_threshold: float = 0.1,
                warmup_steps: int = 100):    # <── NEW PARAM

        self.sensor_id = sensor_id
        self.window_length = window_length
        self.threshold_k = threshold_k
        self.drift_threshold = drift_threshold
        self.warmup_steps = warmup_steps

        # Model & metadata
        self.model = None
        self.scaler = None
        self.is_model_loaded = False

        # Buffers
        self.error_memory = deque(maxlen=memory_size)
        self.data_memory = deque(maxlen=memory_size)
        self.recent_errors = deque(maxlen=100)

        # Rolling stats
        self.rolling_stats = {
            'median': 0.0,
            'mad': 1.0,
            'mean': 0.0,   # backward compatibility
            'std': 1.0,    # backward compatibility
            'q95': 0.0,
            'q99': 0.0
        }
        self.baseline_errors = None

        # Counters
        self.total_processed = 0
        self.anomalies_detected = 0
        self.drift_detected_count = 0
        self.last_stats_update = datetime.now()

        self.anomaly_cooldown = 0
        self.drift_cooldown = 0

        self.anomaly_cooldown_steps = 5    # you can tune
        self.drift_cooldown_steps = 10     # you can tune

        self.consecutive_drift_votes = 0
        self.consecutive_anomaly_votes = 0

        if model_path:
            self.load_model(model_path)

    def load_model(self, model_path: str) -> bool:
        """Load pretrained AE model + metadata."""
        try:
            if KERAS_AVAILABLE and model_path.endswith('.h5'):
                self.model = load_model(model_path, compile=False)

                # Correct metadata file
                metadata_path = model_path.replace('_model.h5', '_metadata.pkl')

                if os.path.exists(metadata_path):
                    with open(metadata_path, 'rb') as f:
                        metadata = pickle.load(f)

                    baseline = metadata.get('baseline_stats', None)

                    if baseline is not None:
                        # Initialize rolling stats from training
                        # Load robust baseline stats
                        self.rolling_stats['median'] = baseline.get('median')
                        self.rolling_stats['mad']    = baseline.get('mad')

                        # Backward compatibility for other parts of system
                        self.rolling_stats['mean'] = self.rolling_stats['median']
                        self.rolling_stats['std']  = self.rolling_stats['mad']

                        self.rolling_stats['q95']  = baseline['q95']
                        self.rolling_stats['q99']  = baseline['q99']

                        # Save baseline distribution for drift detection
                        self.baseline_errors = np.array(baseline['baseline_errors'])

                # AE was trained on raw, NOT scaled
                self.scaler = None

            else:
                raise ValueError("Unsupported model format – expecting .h5 AE model")

            self.is_model_loaded = True
            print(f"✅ AE model loaded for sensor {self.sensor_id}")
            return True

        except Exception as e:
            print(f"❌ Failed to load AE model for sensor {self.sensor_id}: {e}")
            return False


    def observe(self, sensor_subsequence: np.ndarray) -> Dict:
        """Observe subsequence [window_length] and return anomaly/drift flags."""
        if not self.is_model_loaded:
            return {"sensor_id": self.sensor_id, "error": "no_model_loaded", "timestamp": datetime.now()}

        if len(sensor_subsequence) != self.window_length:
            return {"sensor_id": self.sensor_id,
                    "error": f"invalid_length_expected_{self.window_length}_got_{len(sensor_subsequence)}",
                    "timestamp": datetime.now()}

        # 1. Anomaly score
        anomaly_score = self._compute_robust_anomaly_score(sensor_subsequence)

        # 2. Update memory
        self.data_memory.append(sensor_subsequence.copy())
        self.error_memory.append(anomaly_score)
        self.recent_errors.append(anomaly_score)

        # 3

        # --------------- WARM-UP PHASE -----------------
        # During warm-up, rolling stats ignore live data and stay fixed
        if self.total_processed < self.warmup_steps:
            med = np.median(self.baseline_errors)
            mad = np.median(np.abs(self.baseline_errors - med)) + 1e-8

            self.rolling_stats['median'] = med
            self.rolling_stats['mad'] = mad
            self.rolling_stats['mean'] = med     # backward compatibility
            self.rolling_stats['std'] = mad
        else:
            # After warm-up, rolling stats evolve normally
            if len(self.error_memory) >= 50 and len(self.error_memory) % 10 == 0:
                self._update_rolling_stats(list(self.error_memory)[-50:])

        # 4. Flags
        is_anomaly = self._check_adaptive_anomaly(anomaly_score)
        drift_flag = self._check_advanced_drift()
        needs_retrain = self._check_retrain_need()
        confidence = self._compute_robust_confidence(anomaly_score)

        # 5. Update counters
        self.total_processed += 1
        if is_anomaly: self.anomalies_detected += 1
        if drift_flag: self.drift_detected_count += 1

        return {
            "sensor_id": self.sensor_id,
            "timestamp": datetime.now(),
            "is_anomaly": bool(is_anomaly),
            "drift_flag": bool(drift_flag),
            "needs_retrain_flag": bool(needs_retrain),
            "anomaly_score": float(anomaly_score),
            "confidence": float(confidence),
            "threshold_used": float(self.rolling_stats['median'] + self.threshold_k * self.rolling_stats['mad']),
            "anomaly_rate": self.anomalies_detected / max(1, self.total_processed),
            "drift_rate": self.drift_detected_count / max(1, self.total_processed)
        }

    def _compute_robust_anomaly_score(self, subsequence: np.ndarray) -> float:
        """Compute reconstruction error using AE model on RAW values."""
        try:
            # Ensure shape: [1, window_length, 1]
            X = subsequence.reshape(1, self.window_length, 1)
            reconstruction = self.model.predict(X, verbose=0)

            error = mean_squared_error(
                subsequence.flatten(),
                reconstruction.flatten()
            )
            return max(0.0, error)

        except Exception as e:
            print(f"⚠️ AE inference failed for sensor {self.sensor_id}: {e}")
            # Fallback: variance of raw subsequence
            return float(np.var(subsequence))

    def _update_rolling_stats(self, errors: List[float]):
        errors_array = np.array(errors)

        median = np.median(errors_array)
        mad = np.median(np.abs(errors_array - median)) + 1e-8  # avoid zero

        # Store
        self.rolling_stats['median'] = median
        self.rolling_stats['mad'] = mad

        # Backward compatibility fields (for plotting)
        self.rolling_stats['mean'] = median
        self.rolling_stats['std'] = mad

        # Percentile bands (unchanged; good for drift & visualization)
        self.rolling_stats['q95'] = np.percentile(errors_array, 95)
        self.rolling_stats['q99'] = np.percentile(errors_array, 99)

        self.last_stats_update = datetime.now()

    def _check_adaptive_anomaly(self, score: float) -> bool:
        median = self.rolling_stats.get('median', self.rolling_stats['mean'])
        mad = self.rolling_stats.get('mad', self.rolling_stats['std'])
        threshold = median + self.threshold_k * mad
        is_anomaly_now = score > threshold

        # Cooldown active → suppress anomaly
        if self.anomaly_cooldown > 0:
            self.anomaly_cooldown -= 1
            return False

        # No cooldown and anomaly happened → activate cooldown
        if is_anomaly_now:
            self.anomaly_cooldown = self.anomaly_cooldown_steps
            return True

        return False

    def _check_advanced_drift(self) -> bool:
        if self.baseline_errors is None or len(self.recent_errors) < 30:
            return False
        try:
            hist_baseline, bins = np.histogram(self.baseline_errors, bins=20, density=True)
            hist_recent, _ = np.histogram(list(self.recent_errors), bins=bins, density=True)
            hist_baseline += 1e-10; hist_recent += 1e-10
            hist_baseline /= hist_baseline.sum(); hist_recent /= hist_recent.sum()
            js_divergence = jensenshannon(hist_baseline, hist_recent)
            is_drift_now = js_divergence > self.drift_threshold

            # Cooldown active → suppress
            if self.drift_cooldown > 0:
                self.drift_cooldown -= 1
                return False

            # Multi-step confirmation: require 3 drift votes in last few steps
            if is_drift_now:
                self.consecutive_drift_votes += 1
            else:
                self.consecutive_drift_votes = 0

            if self.consecutive_drift_votes >= 3:
                self.drift_cooldown = self.drift_cooldown_steps
                self.consecutive_drift_votes = 0
                return True

            return False

        except Exception:
            try:
                _, p_value = stats.ks_2samp(self.baseline_errors, list(self.recent_errors))
                return p_value < 0.05
            except:
                return False

    def _check_retrain_need(self) -> bool:
        if len(self.error_memory) < 100: return False
        recent_errors = list(self.error_memory)[-50:]
        threshold = self.rolling_stats['mean'] + self.threshold_k * self.rolling_stats['std']
        anomaly_rate = sum(1 for e in recent_errors if e > threshold) / len(recent_errors)
        criteria = [
            anomaly_rate > 0.3,
            self.drift_detected_count > 0.1 * self.total_processed,
            np.mean(recent_errors) > 2.0 * self.rolling_stats['mean'] if len(recent_errors) > 0 else False,
            (datetime.now() - self.last_stats_update).days > 7
        ]
        return sum(criteria) >= 2

    def _compute_robust_confidence(self, score: float) -> float:
        median = self.rolling_stats.get('median')
        mad = self.rolling_stats.get('mad')

        if mad == 0:
            return 0.5

        threshold = median + self.threshold_k * mad

        z = (score - threshold) / mad  # how far beyond threshold?

        # Smooth probability-like mapping
        confidence = 1 / (1 + np.exp(-z))

        return float(np.clip(confidence, 0.0, 1.0))



# =====================================================
# ROBUST MASTER AGENT
# =====================================================

class RobustMasterAgent:
    """Aggregates sensor results, makes system-level anomaly/drift/retrain decisions."""
    def __init__(self, sensor_agents: List[RobustSensorAgent],
                 system_anomaly_threshold: float = 0.3,
                 drift_threshold: float = 0.2,
                 retrain_threshold: float = 0.15):
        self.sensor_agents = sensor_agents
        self.num_sensors = len(sensor_agents)
        self.system_anomaly_threshold = system_anomaly_threshold
        self.drift_threshold = drift_threshold
        self.retrain_threshold = retrain_threshold

    def process_system_input(self, system_subsequence: np.ndarray) -> Dict:
        """Process [window_length, num_sensors] multivariate subsequence."""
        timestamp = datetime.now()
        if system_subsequence.shape[1] != self.num_sensors:
            return {"error": f"Expected {self.num_sensors} sensors, got {system_subsequence.shape[1]}",
                    "timestamp": timestamp}

        # 1. Collect sensor observations
        sensor_results = []
        for i, agent in enumerate(self.sensor_agents):
            sensor_data = system_subsequence[:, i]
            result = agent.observe(sensor_data)
            sensor_results.append(result)

        # 2. Simple aggregation
        anomalies = sum(1 for r in sensor_results if r.get("is_anomaly"))
        drifts = sum(1 for r in sensor_results if r.get("drift_flag"))
        retrains = sum(1 for r in sensor_results if r.get("needs_retrain_flag"))

        anomaly_rate = anomalies / max(1, self.num_sensors)
        drift_rate = drifts / max(1, self.num_sensors)
        retrain_rate = retrains / max(1, self.num_sensors)

        system_decisions = {
            "system_anomaly": anomaly_rate >= self.system_anomaly_threshold,
            "system_drift": drift_rate >= self.drift_threshold,
            "system_needs_retrain": retrain_rate >= self.retrain_threshold,
            "anomaly_rate": anomaly_rate,
            "drift_rate": drift_rate,
            "retrain_rate": retrain_rate
        }

        return {
            "timestamp": timestamp,
            "sensor_results": sensor_results,
            "system_decisions": system_decisions
        }



# =====================================================
# SYSTEM CREATION
# =====================================================

def create_robust_system(num_sensors: int, models_dir: str, win_length: int, warmup_steps: int = 100) -> Tuple[List[RobustSensorAgent], RobustMasterAgent]:
    """Create robust sensor system loading AE models + metadata."""
    print(f"🚀 Creating robust system with {num_sensors} sensors")
    sensor_agents = []
    for sensor_id in range(num_sensors):
        model_path = os.path.join(models_dir, f"sensor_{sensor_id}_model.h5")
        agent = RobustSensorAgent(sensor_id=sensor_id,
                                  model_path=model_path if os.path.exists(model_path) else None,
                                  window_length=win_length,
                                  memory_size=1000,
                                  threshold_k=2.0,
                                  drift_threshold=0.1,
                                  warmup_steps=warmup_steps)       # <── NEW
        sensor_agents.append(agent)

    master = RobustMasterAgent(sensor_agents=sensor_agents,
                               system_anomaly_threshold=0.3,
                               drift_threshold=0.2,
                               retrain_threshold=0.15)
    print(f"✅ Created system: {len([a for a in sensor_agents if a.is_model_loaded])}/{num_sensors} models loaded")

    return sensor_agents, master



#=========================================================================================================================================
#=================  MAIN PERCEPTION LAYER PRECOMPUTATION CODE========================================================================================================================
#=========================================================================================================================================
#=========================================================================================================================================
#############################################
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import joblib


    # -------------------------------------------------
    # 1. LOAD DATA + MASK + LABELS
    # -------------------------------------------------
data_path       = "/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/multivariate_long_sequences-TRAIN-10Sec-DIRECT-VAR.npy"
label_path      = "/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/window_labels_3class.npy"
train_mask_path = "/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/train_mask.npy"
test_mask_path  = "/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/test_mask.npy"
holdout_mask_path = "/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/holdout_mask.npy"

X = np.load(data_path)        # (N, W, S)
y = np.load(label_path)       # (N,) ∈ {0,1,2}

train_mask   = np.load(train_mask_path).astype(bool)
test_mask    = np.load(test_mask_path).astype(bool)
holdout_mask = np.load(holdout_mask_path).astype(bool)

# Sanity check (VERY important) -- This assertion enforces strict mutual exclusivity between evaluation and holdout subsets, preventing data leakage and ensuring unbiased performance estimation.
assert not np.any(train_mask & test_mask)
assert not np.any(train_mask & holdout_mask)
assert not np.any(test_mask & holdout_mask)

X_train, y_train = X[train_mask], y[train_mask]
X_test,  y_test  = X[test_mask],  y[test_mask]
X_hold,  y_hold  = X[holdout_mask], y[holdout_mask]

print("Train:", X_train.shape)
print("Test :", X_test.shape)
print("Hold :", X_hold.shape)


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np



#------------------------------------------------------------------------------------------
# -------------TRansformer for 3-class classification - NO CONTRASTIVE LEARNING
#----------------------------------------------------------------------------------------

import os
from tensorflow.keras.models import load_model

MODEL_DIR  = "/content/drive/MyDrive/PHD/2025/models/transformer_3class"
MODEL_PATH = os.path.join(MODEL_DIR, "transformer_classifier.keras")


def transformer_encoder(x, head_size, num_heads, ff_dim, dropout=0.2):
    # --- Self-attention block ---
    attn_output = layers.MultiHeadAttention(
        key_dim=head_size,
        num_heads=num_heads,
        dropout=dropout
    )(x, x)

    attn_output = layers.Dropout(dropout)(attn_output)
    x = layers.Add()([x, attn_output])
    x = layers.LayerNormalization(epsilon=1e-6)(x)

    # --- Feed-forward block ---
    ff_output = layers.Dense(ff_dim, activation="relu")(x)
    ff_output = layers.Dropout(dropout)(ff_output)
    ff_output = layers.Dense(x.shape[-1])(ff_output)

    x = layers.Add()([x, ff_output])
    x = layers.LayerNormalization(epsilon=1e-6)(x)

    return x

def build_transformer(window_len, n_features, n_classes=3):
    inputs = keras.Input(shape=(window_len, n_features))

    # Project features to model dimension
    x = layers.Dense(64)(inputs)

    # Transformer blocks
    x = transformer_encoder(x, head_size=64, num_heads=4, ff_dim=128, dropout=0.2)
    x = transformer_encoder(x, head_size=64, num_heads=4, ff_dim=128, dropout=0.2)

    # Pooling + classifier
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(n_classes, activation="softmax")(x)

    model = keras.Model(inputs, outputs)
    return model

from sklearn.utils.class_weight import compute_class_weight
n_classes = 3

if os.path.exists(MODEL_PATH):
    print("✅ Found pretrained Transformer. Loading...")

    classifier = load_model(MODEL_PATH)
    classifier.trainable = False

else:
    print("🚀 No pretrained model found. Training Transformer...")

    window_len  = X_train.shape[1]
    n_features = X_train.shape[2]

    classifier = build_transformer(window_len, n_features, n_classes)

    classifier.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            keras.metrics.SparseTopKCategoricalAccuracy(k=2, name="top2_acc"),
        ]
    )

    # Class weights (IMBALANCE HANDLING)
    class_weights = compute_class_weight(
        class_weight="balanced",
        classes=np.unique(y_train),
        y=y_train
    )

    class_weight = {
        i: w for i, w in zip(np.unique(y_train), class_weights)
    }

    early_stop = keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=2,
        restore_best_weights=True
    )

    history = classifier.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=50,
        batch_size=256,
        class_weight=class_weight,
        callbacks=[early_stop],
        verbose=2
    )

    os.makedirs(MODEL_DIR, exist_ok=True)
    classifier.save(MODEL_PATH, include_optimizer=False)

    print("✅ Transformer trained and saved.")



####-----------------------------Load and test-----------------------------

import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.metrics import classification_report, confusion_matrix


def focal_loss(gamma=2.0, alpha=0.25):
    def loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())

        bce = K.binary_crossentropy(y_true, y_pred)
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)

        alpha_factor = y_true * alpha + (1 - y_true) * (1 - alpha)
        modulating_factor = K.pow(1.0 - p_t, gamma)

        return K.mean(alpha_factor * modulating_factor * bce)
    return loss


model_path = "/content/drive/MyDrive/PHD/2025/models/transformer_3class/transformer_classifier.keras"

from tensorflow.keras.models import load_model

classifier = load_model(
    model_path,
    custom_objects={"focal_loss": focal_loss}  # only if you actually used it
)

classifier.trainable = False  # safety


print("✅ Transformer loaded for TEST")
probs_test = classifier.predict(X_test)
y_pred_test = np.argmax(probs_test, axis=1)

print("\n=== TEST SET PERFORMANCE ===")
print(classification_report(y_test, y_pred_test, digits=4))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred_test))

###-------------------------HOLDOUT-------------------
############################################################
probs_hold = classifier.predict(X_hold)
y_pred_hold = np.argmax(probs_hold, axis=1)

print("\n=== HOLDOUT SET PERFORMANCE (FINAL) ===")
print(classification_report(y_hold, y_pred_hold, digits=4))
print("Confusion matrix:\n", confusion_matrix(y_hold, y_pred_hold))



def transformer_signals(classifier, X_window):
    probs = classifier.predict(X_window, verbose=0)[0]  # shape (3,)

    p0, p1, p2 = probs

    return {
        "p_normal": float(p0),
        "early_fault_score": float(p1),
        "severe_fault_score": float(p2),
        "anomaly_score": float(1.0 - p0)
    }

signals = transformer_signals(classifier, X_hold) #Get actual prob distribution now
print(signals)

# ============================================================
# HOLDOUT PERCEPTION PRECOMPUTE (CHECKPOINTED + RESUMABLE)
# ============================================================

import os
import json
import numpy as np
from tqdm import tqdm

def json_safe(obj):
    import numpy as np
    from collections import deque

    if isinstance(obj, (np.integer,)):
        return int(obj)
    if isinstance(obj, (np.floating,)):
        return float(obj)
    if isinstance(obj, (np.ndarray,)):
        return obj.tolist()
    if isinstance(obj, deque):
        return list(obj)
    if isinstance(obj, dict):
        return {k: json_safe(v) for k, v in obj.items()}
    if isinstance(obj, list):
        return [json_safe(v) for v in obj]
    if isinstance(obj, tuple):
        return [json_safe(v) for v in obj]
    if obj is None or isinstance(obj, (str, int, float, bool)):
        return obj

    # last resort (custom objects)
    return str(obj)


# --------------------------------------------------
# CONFIG
# --------------------------------------------------
CACHE_PATH = "/content/drive/MyDrive/PHD/2025/cache/holdout_precomp.json"
TMP_PATH   = CACHE_PATH + ".tmp"

SAVE_EVERY   = 250          # checkpoint frequency
MAX_SAMPLES  = 2000         # None = full holdout, or set to 5000, 10000, etc.

# --------------------------------------------------
# ASSUMED IN MEMORY
# --------------------------------------------------
# X_hold, y_hold
# classifier
# create_robust_system
# AdaptiveWindowAgent

models_dir = "/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/sensor/model"


# --------------------------------------------------
# RESUME OR INIT
# --------------------------------------------------
if os.path.exists(CACHE_PATH):
    print("♻️ Resuming existing HOLDOUT precompute...")
    with open(CACHE_PATH, "r") as f:
        PRECOMP = json.load(f)

    done_indices = {item["index"] for item in PRECOMP}
    start_idx = max(done_indices) + 1 if done_indices else 0
else:
    print("🆕 Starting fresh HOLDOUT precompute...")
    PRECOMP = []
    done_indices = set()
    start_idx = 0


# --------------------------------------------------
# LIMIT
# --------------------------------------------------
N_TOTAL = len(X_hold)
END_IDX = N_TOTAL if MAX_SAMPLES is None else min(N_TOTAL, MAX_SAMPLES)

print(f"➡️ Starting at index {start_idx}, ending at {END_IDX}")
print(f"📦 Already cached: {len(PRECOMP)} samples")


# --------------------------------------------------
# BUILD PERCEPTION AGENTS ONCE
# --------------------------------------------------
num_sensors = X_hold.shape[2]
win_length  = X_hold.shape[1]

sensor_agents, master = create_robust_system(
    num_sensors=num_sensors,
    models_dir=models_dir,
    win_length=win_length,
    warmup_steps=100,
)

window_agent = AdaptiveWindowAgent(
    model_path="/content/drive/MyDrive/PHD/2025/DGRNet-MLP-Versions/METROPM_MLP_model_10Sec.keras"
)


# --------------------------------------------------
# MAIN LOOP
# --------------------------------------------------
print("\n⏳ Running HOLDOUT perception precompute...\n")

for i in tqdm(range(start_idx, END_IDX)):
    if i in done_indices:
        continue

    seq = X_hold[i]

    try:
        # -------------------------
        # Perception
        # -------------------------
        master_out = master.process_system_input(seq)
        window_out = window_agent.predict_window_size(seq.flatten(), seq)

        probs = classifier.predict(seq[np.newaxis, ...], verbose=0)[0]
        p0, p1, p2 = probs

        PRECOMP.append({
            "index": i,
            "y_true": int(y_hold[i]),
            "master": master_out,
            "window": window_out,
            "model_outputs": {
                "p_normal": float(p0),
                "p_warn": float(p1),
                "p_fault": float(p2),
                "argmax": int(np.argmax(probs)),
            }
        })

    except Exception as e:
        print(f"⚠️ Failed at index {i}: {e}")
        continue

    # -------------------------
    # CHECKPOINT
    # -------------------------
    if (i + 1) % SAVE_EVERY == 0:
        with open(TMP_PATH, "w") as f:
            json.dump(json_safe(PRECOMP), f)


        os.replace(TMP_PATH, CACHE_PATH)
        print(f"💾 Checkpoint saved at {i+1}/{END_IDX}")

# --------------------------------------------------
# FINAL SAVE
# --------------------------------------------------
with open(TMP_PATH, "w") as f:
    json.dump(PRECOMP, f)
os.replace(TMP_PATH, CACHE_PATH)

print("\n✅ HOLDOUT precompute complete")
print(f"📦 Total cached samples: {len(PRECOMP)}")
print(f"📁 Saved to: {CACHE_PATH}")



🚀 Creating robust system with 12 sensors
✅ AE model loaded for sensor 0
✅ AE model loaded for sensor 1
✅ AE model loaded for sensor 2
✅ AE model loaded for sensor 3
✅ AE model loaded for sensor 4
✅ AE model loaded for sensor 5
✅ AE model loaded for sensor 6
✅ AE model loaded for sensor 7
✅ AE model loaded for sensor 8
✅ AE model loaded for sensor 9
✅ AE model loaded for sensor 10
✅ AE model loaded for sensor 11
✅ Created system: 12/12 models loaded
✅ Loaded baseline error + window distribution.
   Error median=0.001153, MAD=0.000440
   Window mean=67.258, std=22.841
✅ Loaded MLP model from /content/drive/MyDrive/PHD/2025/DGRNet-MLP-Versions/METROPM_MLP_model_10Sec.keras
AdaptiveWindowAgent adaptive_window_agent initialized
✅ Loaded NSP LSTM next-step predictor
🆕 Starting fresh holdout evaluation


KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')